In [3]:
import xarray as xr
import glob

files = sorted(glob.glob("../1_DATA/1_RAW/GFED5.1_monthly_*.nc"))

ds0 = xr.open_dataset(files[0], engine="netcdf4")
print(list(ds0.data_vars))


['DM', 'C', 'CO2', 'CO', 'CH4', 'NMOC_g', 'H2', 'NOx', 'N2O', 'PM2.5', 'TPC', 'OC', 'BC', 'SO2', 'C2H6', 'CH3OH', 'C2H5OH', 'C3H8', 'C2H2', 'C2H4', 'C3H6', 'C5H8', 'C10H16', 'C7H8', 'C6H6', 'C8H10', 'Toluene_Lump', 'Higher_Alkenes', 'Higher_Alkanes', 'CH2O', 'C2H4O', 'C3H6O', 'NH3', 'C2H6S', 'HCN', 'HCOOH', 'CH3COOH', 'MEK', 'CH3COCHO', 'HOCH2CHO']


In [7]:
DM_yearly_grid = ds0["DM"].resample(time="YS").sum()

# replace time with a clean "year" dimension
DM_yearly_grid = (
    DM_yearly_grid
    .assign_coords(year=DM_yearly_grid.time.dt.year)
    .swap_dims({"time": "year"})
    .drop_vars("time")
)

DM_yearly_grid


<xarray.DataArray 'DM' (year: 1, lat: 180, lon: 360)> Size: 259kB
array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]],
      shape=(1, 180, 360), dtype=float32)
Coordinates:
  * year     (year) int64 8B 1997
  * lat      (lat) float32 720B -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * lon      (lon) float32 1kB -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
Attributes:
    units:          g DM per month
    standard_name:  fire emissions of DM

In [8]:
DM_yearly_global = ds0["DM"].resample(time="YS").sum().sum(["lat", "lon"])

DM_yearly_global = (
    DM_yearly_global
    .assign_coords(year=DM_yearly_global.time.dt.year)
    .swap_dims({"time": "year"})
    .drop_vars("time")
)

df_global = DM_yearly_global.to_dataframe(name="DM_g_year").reset_index()
df_global.head()


,year,DM_g_year
0,1997,8.194480e+15


In [11]:
print("n_files:", len(files))
print("n_time:", ds0.dims["time"])
print("time min/max:", str(ds0.time.min().values), str(ds0.time.max().values))


n_files: 26
n_time: 12
time min/max: 1997-01-01T01:00:00.000000000 1997-12-01T01:00:00.000000000


C:\Users\zgcha\AppData\Local\Temp\ipykernel_67904\1546805418.py:2: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  print("n_time:", ds0.dims["time"])


In [13]:
import xarray as xr
import numpy as np
import glob

files = sorted(glob.glob("../1_DATA/1_RAW/GFED5.1_monthly_*.nc"))

def preprocess(ds):
    # normalize lon to [-180, 180) + round to avoid float mismatch across files
    lon = ds["lon"].astype("float64")
    lon_fixed = ((lon + 180) % 360) - 180
    lon_fixed = np.round(lon_fixed, 6)

    ds = ds.assign_coords(lon=lon_fixed).sortby("lon").sortby("lat")
    return ds

ds_all = xr.open_mfdataset(
    files,
    engine="netcdf4",
    combine="nested",
    concat_dim="time",
    preprocess=preprocess,
    coords="minimal",
    data_vars="minimal",
    compat="override",
    parallel=True,
    chunks={"time": 120},
).sortby("time")

print(ds_all.time.min().values, ds_all.time.max().values, ds_all.dims["time"])


C:\Users\zgcha\AppData\Local\Temp\ipykernel_67904\2590964534.py:16: FutureWarning: In a future version of xarray the default value for join will change from join='outer' to join='exact'. This change will result in the following ValueError: cannot be aligned with join='exact' because index/labels/sizes are not equal along these coordinates (dimensions): 'lat' ('lat',) The recommendation is to set join explicitly for this case.
  ds_all = xr.open_mfdataset(
C:\Users\zgcha\AppData\Local\Temp\ipykernel_67904\2590964534.py:16: FutureWarning: In a future version of xarray the default value for join will change from join='outer' to join='exact'. This change will result in the following ValueError: cannot be aligned with join='exact' because index/labels/sizes are not equal along these coordinates (dimensions): 'lon' ('lon',) The recommendation is to set join explicitly for this case.
  ds_all = xr.open_mfdataset(


1997-01-01T01:00:00.000000000 2022-12-01T01:00:00.000000000 312


C:\Users\zgcha\AppData\Local\Temp\ipykernel_67904\2590964534.py:29: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  print(ds_all.time.min().values, ds_all.time.max().values, ds_all.dims["time"])


In [16]:
DM_yearly_global = ds_all["DM"].resample(time="YS").sum().sum(["lat", "lon"])

DM_yearly_global = (
    DM_yearly_global
    .assign_coords(year=DM_yearly_global.time.dt.year)
    .swap_dims({"time": "year"})
    .drop_vars("time")
)

df_global = DM_yearly_global.to_dataframe(name="DM_g_year").reset_index()
df_global.head(), df_global.tail()


(   year     DM_g_year
 0  1997  8.194480e+15
 1  1998  9.306521e+15
 2  1999  7.159083e+15
 3  2000  7.238705e+15
 4  2001  6.977689e+15,
     year     DM_g_year
 21  2018  6.175737e+15
 22  2019  7.524318e+15
 23  2020  7.387265e+15
 24  2021  7.091344e+15
 25  2022  6.060949e+15)

In [17]:
df_global.to_csv('DM_Burned_97_22.csv')

In [25]:
import xarray as xr
import numpy as np
import pandas as pd

ds_tau = xr.open_dataset("../1_DATA/1_RAW/tau_map_2012-12.nc")

tau = ds_tau["tau"]  # (month, lat)

# area-weighted global mean across lat
weights = np.cos(np.deg2rad(ds_tau["lat"]))
tau_global_monthly = tau.weighted(weights).mean("lat")  # (month,)

# Group by calendar year
g = tau_global_monthly.groupby("month.year")

# Yearly stats
df_mean = g.mean("month").to_dataframe(name="tau_global_mean").reset_index()
df_max  = g.max("month").to_dataframe(name="tau_global_max").reset_index()
df_var  = g.var("month").to_dataframe(name="tau_global_var").reset_index()

# Merge
df_tau_yearly = (
    df_mean
    .merge(df_max, on="year", how="inner")
    .merge(df_var, on="year", how="inner")
    .sort_values("year")
)

# Export
out_path = "../1_DATA/2_PROCESSED/tau_yearly_stats.csv"
df_tau_yearly.to_csv(out_path, index=False)

df_tau_yearly.tail()


,year,tau_global_mean,tau_global_max,tau_global_var
158,2008,0.003774,0.004774,3.579604e-07
159,2009,0.004662,0.006178,8.586058e-07
160,2010,0.003555,0.004473,1.273640e-07
161,2011,0.004931,0.006751,1.609619e-06
162,2012,0.002809,0.004844,2.774706e-06


In [38]:
import pandas as pd

df = pd.read_csv("../1_DATA/1_RAW/nino34.long.anom.csv")

# Clean column names
df.columns = [c.strip() for c in df.columns]

# Keep + rename
df = df[["Date", "NINA34"]].dropna().rename(columns={"NINA34": "nino34"})

# ---- Robust Date parsing for mixed formats ----
# 1) Try default parser (handles YYYY-MM-DD, many ISO-ish strings)
df["Date_parsed"] = pd.to_datetime(df["Date"], errors="coerce")

# 2) For things like 6/1/1900 (could be mm/dd/yyyy or dd/mm/yyyy):
#    Try US-style first (month/day/year), then day-first fallback.
mask = df["Date_parsed"].isna()
if mask.any():
    df.loc[mask, "Date_parsed"] = pd.to_datetime(df.loc[mask, "Date"], errors="coerce", dayfirst=False)

mask = df["Date_parsed"].isna()
if mask.any():
    df.loc[mask, "Date_parsed"] = pd.to_datetime(df.loc[mask, "Date"], errors="coerce", dayfirst=True)

# 3) For compact YYYYMM numeric strings (e.g., 190006)
mask = df["Date_parsed"].isna()
if mask.any():
    df.loc[mask, "Date_parsed"] = pd.to_datetime(
        df.loc[mask, "Date"].astype(str),
        format="%Y%m",
        errors="coerce"
    )

# Finalize
df = df.dropna(subset=["Date_parsed"]).drop(columns=["Date"]).rename(columns={"Date_parsed": "Date"})
df = df.sort_values("Date")

# Yearly stats
df["year"] = df["Date"].dt.year
df_nino_yearly = (
    df.groupby("year")["nino34"]
      .agg(nino34_mean="mean", nino34_max="max", nino34_var="var")
      .reset_index()
      .sort_values("year")
)

out_path = "../1_DATA/2_PROCESSED/nino34_yearly_stats.csv"
df_nino_yearly.to_csv(out_path, index=False)

df_nino_yearly.head()


,year,nino34_mean,nino34_max,nino34_var
0,1870,-0.928333,-0.53,0.041288
1,1871,-0.449167,-0.24,0.024154
2,1872,-0.712500,-0.32,0.041457
3,1873,-0.695000,-0.34,0.073082
4,1874,-1.067500,-0.72,0.038711


In [40]:
df_nino_yearly.to_csv('../1_DATA/2_PROCESSED/NINA_NINO_yearly.csv')


In [42]:
import xarray as xr

ds_pdsi = xr.open_dataset("../1_DATA/1_RAW/scPDSI.cru_ts4.09early1.1901.2024.cal_1901_24.bams.2025.GLOBAL.IGBP.WHC.1901.2024.nc")  # <-- adjust filename
print(ds_pdsi)
print("data_vars:", list(ds_pdsi.data_vars))
print("coords:", list(ds_pdsi.coords))


<xarray.Dataset> Size: 3GB
Dimensions:    (time: 1488, latitude: 360, longitude: 720)
Coordinates:
  * time       (time) datetime64[ns] 12kB 1901-01-01 1901-02-01 ... 2024-12-01
  * latitude   (latitude) float64 3kB 89.75 89.25 88.75 ... -88.75 -89.25 -89.75
  * longitude  (longitude) float64 6kB -179.8 -179.2 -178.8 ... 179.2 179.8
Data variables:
    scpdsi     (time, latitude, longitude) float64 3GB ...
Attributes:
    creation_date:                             11-Mar-2025 16:38:46
    Institution:                               Climatic Research Unit, Univer...
    creator:                                   Prof. Jonathan Barichivich (jo...
    Original reference for computation setup:  van der Schrier G, Barichivich...
    Updated reference:                         J. Barichivich, Osborn TJ, Har...
data_vars: ['scpdsi']
coords: ['longitude', 'latitude', 'time']


In [ ]:

pdsi = ds_pdsi["scpdsi"]

# yearly stats across months
pdsi_y_mean = pdsi.resample(time="YS").mean("time")
pdsi_y_med  = pdsi.resample(time="YS").median("time")
pdsi_y_max  = pdsi.resample(time="YS").max("time")
pdsi_y_var  = pdsi.resample(time="YS").var("time")

# convert time -> year dimension
def to_year_dim(da):
    return (da.assign_coords(year=da.time.dt.year)
              .swap_dims({"time": "year"})
              .drop_vars("time"))

ds_yearly = xr.Dataset({
    "scpdsi_year_mean": to_year_dim(pdsi_y_mean),
    "scpdsi_year_median": to_year_dim(pdsi_y_med),
    "scpdsi_year_max": to_year_dim(pdsi_y_max),
    "scpdsi_year_var": to_year_dim(pdsi_y_var),
})

# write to disk (netcdf)
out_nc = "../1_DATA/2_PROCESSED/scPDSI_yearly_grid_stats.nc"
ds_yearly.to_netcdf(out_nc)
print("Saved:", out_nc)

NameError: name 'ds' is not defined

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr

path = "../1_DATA/1_RAW/scPDSI_1901_2024.nc"  # adjust to your filename
ds = xr.open_dataset(path, chunks={"time": 120, "latitude": 90, "longitude": 180})

pdsi = ds["scpdsi"]



weights = np.cos(np.deg2rad(pdsi["latitude"]))
pdsi_global_monthly = pdsi.weighted(weights).mean("latitude").mean("longitude")

g = pdsi_global_monthly.groupby("time.year")
df_global = xr.Dataset({
    "scpdsi_global_mean": g.mean("time"),
    "scpdsi_global_median": g.median("time"),
    "scpdsi_global_max": g.max("time"),
    "scpdsi_global_var": g.var("time"),
}).to_dataframe().reset_index().sort_values("year")

out_csv = "../1_DATA/2_PROCESSED/scPDSI_yearly_global_stats.csv"
df_global.to_csv(out_csv, index=False)
df_global.tail()


,year,scpdsi_global_mean,scpdsi_global_median,scpdsi_global_max,scpdsi_global_var
119,2020,-0.169439,-0.167494,0.003698,0.011894
120,2021,-0.254240,-0.271730,-0.054080,0.009646
121,2022,-0.103279,-0.071841,0.037012,0.007309
122,2023,-0.256267,-0.309398,0.037036,0.027012
123,2024,-0.046479,-0.075791,0.107930,0.011201
